In [23]:
import tensorflow as tf
from reader import create_inputs
import json

In [24]:
param_filename = "recog_params.json"
with open(param_filename, 'r') as f:
        recog_params = json.load(f)

In [3]:
image, label = create_inputs(input_channel=3,
                             dilations=recog_params['cnn']['dilations'])

In [4]:
queue = tf.FIFOQueue(256, ['uint8', 'string'])
enqueue = queue.enqueue([image, label])
input_data = queue.dequeue()

In [5]:
sess = tf.Session()

In [6]:
coord = tf.train.Coordinator()
qr = tf.train.QueueRunner(queue, [enqueue])
qr.create_threads(sess, coord=coord, start=True)
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [7]:
dd= sess.run(input_data)

In [11]:
type(recog_params)

dict

In [34]:
import tensorflow as tf
batch_size = 1
time_len = 10
num_classes = 5
labels = tf.constant([0,1,2], tf.int32)
input_data = tf.random_normal([batch_size, time_len, num_classes])
input_data = tf.transpose(input_data, [1, 0, 2])
loss = tf.nn.ctc_loss(input_data, labels, [10])

TypeError: Expected labels to be a SparseTensor

In [2]:
import tensorflow as tf

In [7]:
cells_fw_list = []
cells_bw_list = []
for layer_idx, unit_num in enumerate([200, 200]):
    cells_fw_list.append(tf.nn.rnn_cell.GRUCell(unit_num))
    cells_bw_list.append(tf.nn.rnn_cell.GRUCell(unit_num))
cells_fw = tf.nn.rnn_cell.MultiRNNCell(cells_fw_list)
cells_bw = tf.nn.rnn_cell.MultiRNNCell(cells_bw_list)
current_layer = tf.random_normal([1, 10, 10])
initial_state_fw = tf.random_normal([1, 10])
initial_state_bw = tf.random_normal([1, 10])
length = tf.shape(current_layer)[1]
t = tf.expand_dims(length, 0)
print(t.get_shape())
tf.nn.bidirectional_dynamic_rnn(cell_fw=cells_fw,
                                cell_bw=cells_bw,
                                inputs=current_layer,
                                sequence_length=t,
                                dtype=tf.float32)

(1,)


((<tf.Tensor 'BiRNN/FW/FW/transpose:0' shape=(1, 10, 200) dtype=float32>,
  <tf.Tensor 'ReverseSequence:0' shape=(1, 10, 200) dtype=float32>),
 ((<tf.Tensor 'BiRNN/FW/FW/while/Exit_2:0' shape=(?, 200) dtype=float32>,
   <tf.Tensor 'BiRNN/FW/FW/while/Exit_3:0' shape=(?, 200) dtype=float32>),
  (<tf.Tensor 'BiRNN/BW/BW/while/Exit_2:0' shape=(?, 200) dtype=float32>,
   <tf.Tensor 'BiRNN/BW/BW/while/Exit_3:0' shape=(?, 200) dtype=float32>)))